In [10]:
import pandas as pd
from collections import defaultdict

In [2]:
df = pd.read_csv('data/2016-precinct-president.csv', encoding="ISO-8859-1")
df_ga = df[df['state_postal'] == 'GA']

/Users/lordzamy/anaconda2/envs/py37-analysis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,17,19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_ga

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,...,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
573155,2016,gen,False,Georgia,GA,13,44,Appling County,13001.0,349113.0,...,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
573156,2016,gen,False,Georgia,GA,13,44,Appling County,13001.0,349113.0,...,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
573157,2016,gen,False,Georgia,GA,13,44,Appling County,13001.0,349113.0,...,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
573158,2016,gen,False,Georgia,GA,13,44,Appling County,13001.0,349113.0,...,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
573159,2016,gen,False,Georgia,GA,13,44,Appling County,13001.0,349113.0,...,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605996,2016,gen,False,Georgia,GA,13,44,Worth County,13321.0,343365.0,...,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
605997,2016,gen,False,Georgia,GA,13,44,Worth County,13321.0,343365.0,...,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN
605998,2016,gen,False,Georgia,GA,13,44,Worth County,13321.0,343365.0,...,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN
605999,2016,gen,False,Georgia,GA,13,44,Worth County,13321.0,343365.0,...,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN


In [7]:
candidate_list = list(df_ga['candidate_normalized'].drop_duplicates())
candidate_list

['clinton',
 'johnson',
 'trump',
 'castle',
 'stein',
 'mcmullin',
 'smith',
 'byrne',
 'collins',
 'maturen',
 'buchanan',
 'fox',
 'elliott',
 'hoefling',
 'wilson',
 'kotlikoff',
 'muhammad',
 'cubbler',
 'allen',
 'urbach']

In [8]:
# Create a candidate party map. We will use this to amend the dataframe for null party rows
cand_party_map = {}
for cand in candidate_list:
    party = df_ga['party'][(df_ga['candidate_normalized'] == cand) & (df_ga['party'].isna() == False)].drop_duplicates()
    party_list = list(party)
    if len(party_list) > 0:
        assert len(party_list) == 1
        cand_party_map[cand] = party_list[0]
    else:
        cand_party_map[cand] = 'other'
cand_party_map

{'clinton': 'democratic',
 'johnson': 'libertarian',
 'trump': 'republican',
 'castle': 'other',
 'stein': 'other',
 'mcmullin': 'other',
 'smith': 'other',
 'byrne': 'other',
 'collins': 'other',
 'maturen': 'other',
 'buchanan': 'other',
 'fox': 'other',
 'elliott': 'other',
 'hoefling': 'other',
 'wilson': 'other',
 'kotlikoff': 'other',
 'muhammad': 'other',
 'cubbler': 'other',
 'allen': 'other',
 'urbach': 'other'}

In [11]:
# define vote_map as {county: {precinct: {candidate: int}}}
vote_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for index, row in df_ga.iterrows():
    county = row['county_name'].lower()
    precinct = row['precinct'].lower()
    cand = row['candidate_normalized']
    votes = row['votes']
    
    vote_map[county][precinct][cand] += votes
vote_map

defaultdict(<function __main__.<lambda>()>,
            {'appling county': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'2': defaultdict(int,
                                      {'clinton': 742,
                                       'johnson': 10,
                                       'trump': 427}),
                          'write-ins': defaultdict(int,
                                      {'castle': 3,
                                       'stein': 3,
                                       'mcmullin': 1}),
                          '3c': defaultdict(int,
                                      {'clinton': 154,
                                       'johnson': 14,
                                       'trump': 670}),
                          '4b': defaultdict(int,
                                      {'clinton': 7,
                                       'johnson': 0,
                                       'trump': 305}),
             

In [12]:
import json

In [13]:
# dump candidate party map
with open('data/presidential_candidate_party_map.json', 'w') as f:
    json.dump(cand_party_map, f)

In [14]:
# dump vote map
with open('data/presidential_vote_map.json', 'w') as f:
    json.dump(vote_map, f)